In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
train = pd.read_csv("../input/2021-ml-midterm-p2/train.csv")
test = pd.read_csv("../input/2021-ml-midterm-p2/test.csv")

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
test.head(2)

In [ ]:
# 일단 데이터를 출력해보고, 각각 컬럼마다 unique한 데이터의 개수를 파악

for column in train.columns:
  print(len(train[column].unique()))

In [ ]:
for column in test.columns:
  print(len(test[column].unique()))

In [ ]:
# 유니크한 ID를 뜻하는 컬럼들은 결과와 관련없는 정보로 판단, drop시켜줌
train = pd.read_csv("../input/2021-ml-midterm-p2/train.csv").drop(['index', 'Unnamed: 0', 'customerID'], axis=1) # axis는 열을 의미
test = pd.read_csv("../input/2021-ml-midterm-p2/test.csv").drop(['index', 'Unnamed: 0', 'customerID'], axis=1) # del이용해서 드랍해도 무관함

In [ ]:
X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]
X_test = test

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape) # shape 확인

In [ ]:
X_train.head(2) # 컬럼이 제대로 드랍되었나 확인해봄

In [ ]:
X_test.head(2)

In [ ]:
X_train.info() #데이터의 정보를 봄

In [ ]:
X_test.info()

info를 본 결과 TotalCharges의 타입이 train에서는 object이지만, test에서는 float64인 것을 확인 이에 아래와 같은 과정으로 float타입으로 변경시켜줌

In [ ]:
# import numpy as np
# X_train["TotalCharges"] = pd.to_numeric(X_train["TotalCharges"], downcast="float")

# 위의 주석처리된 명령어를 실행하여
# float타입으로 변경하려 했으나 280번째에 " " 공백이 포함되어 변경이 안되는것을 확인

In [ ]:
(X_train['TotalCharges'] == ' ').sum() # 공백인 곳이 총 10군데 밖에 안되므로 해당 row를 drop하여 처리

In [ ]:
drop_idx = X_train[X_train['TotalCharges'] == ' '].index
print(drop_idx)

In [ ]:
X_train['TotalCharges'][280] # 위에서 구한 index넣어서 실제 값 확인

In [ ]:
X_train = X_train.drop(drop_idx, axis=0) # row를 드랍할것 이므로 axis=0 (디폴트)
y_train = y_train.drop(drop_idx, axis=0)

In [ ]:
X_train["TotalCharges"] = pd.to_numeric(X_train["TotalCharges"], downcast="float") # float타입으로 변경
X_train.info() # 공백 드랍 후, float타입으로 변경이 제대로 되었나 확인

In [ ]:
X_train.head(1) # 이제 라벨인코더 적용필요.

In [ ]:
X_test.head(5)

In [ ]:
X_test.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
columns = ['gender', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
for column in columns:
  le = LabelEncoder()
  X_train[column] = le.fit_transform(X_train[column].values)
  X_test[column] = le.transform(X_test[column].values)

In [ ]:
X_train.head() # 인코딩이 제대로 되었나 확인

In [ ]:
X_test.head()

In [ ]:
for i in X_train.columns:
  X_train.hist(i)
  X_test.hist(i)

In [ ]:
# 스케일러 적용
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
# 1. RobustScaler
sc = RobustScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

# 2. scaler = MinMaxScaler()
# sc = MinMaxScaler()
# X_train_sc = sc.fit_transform(X_train)
# X_test_sc = sc.transform(X_test)

In [ ]:
pd.DataFrame(X_train_sc)

In [ ]:
pd.DataFrame(X_test_sc)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# 1. 로지스틱 회귀
clf = LogisticRegression()
clf.fit(X_train_sc, y_train) # 모델 학습
predict = clf.predict(X_test_sc)

# # 2. LDA
# clf = LinearDiscriminantAnalysis()
# clf.fit(X_train_sc, y_train) # 모델 학습
# predict = clf.predict(X_test_sc)

# 3. QDA 
# clf = QuadraticDiscriminantAnalysis()
# clf.fit(X_train_sc, y_train) # 모델 학습
# predict = clf.predict(X_test_sc)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

In [ ]:
predict

In [ ]:
submit = pd.read_csv("../input/2021-ml-midterm-p2/sample_submit.csv")

In [ ]:
submit['Churn'] = predict

In [ ]:
submit.head(10)

In [ ]:
def cvt_string(data):
  if data.lower()== 'yes' :
    return 1
  elif data.lower() == 'no':
    return 0

In [ ]:
submit['Churn'] = submit['Churn'].apply(cvt_string)

In [ ]:
submit

In [ ]:
submit.to_csv("submit.csv", mode='w', header=True, index= False)

In [ ]:
nan